# Sentiment Classification 

In [5]:
from os import getcwd, chdir
import re
import numpy as np
import pickle as pk
import pandas as pd

from nltk.metrics import ConfusionMatrix
from nltk.classify import NaiveBayesClassifier, MaxentClassifier
from nltk.classify import accuracy
from nltk.tokenize import word_tokenize as wt

from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_selection import SelectKBest
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import chi2
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score

# Change your path here
fpath = getcwd()
print(fpath)

/Users/pierlim/PycharmProjects/sent_mining_CA


In [25]:
df = pd.read_csv('./data/df_reviews.csv')
df.head()

,rating,review,sentiment
0,5,The service at Sushi Bar was great with an ext...,1
1,2,OMG Yelp let me down big time. I don't know h...,-1
2,4,Heard that this was the best sushi joint in Si...,1
3,3,Ebi fry salmon Aburizal ($17.90++) - 3 stars\n...,-1
4,5,Yes I also get the raving reviews now.\nWe dec...,1


In [26]:
# Train-Test Split + Stratify
df.drop(['rating'], inplace=True, axis=1)
df.head()

,review,sentiment
0,The service at Sushi Bar was great with an ext...,1
1,OMG Yelp let me down big time. I don't know h...,-1
2,Heard that this was the best sushi joint in Si...,1
3,Ebi fry salmon Aburizal ($17.90++) - 3 stars\n...,-1
4,Yes I also get the raving reviews now.\nWe dec...,1


In [34]:
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'],
#                                                     stratify=df['sentiment'], 
#                                                     test_size=0.3)
df_train, df_test = train_test_split(df,stratify=df['sentiment'], test_size=0.3)

In [36]:
df_test

,review,sentiment
27,The Sushi Bar is tucked into a little corner o...,1
4,Yes I also get the raving reviews now.\nWe dec...,1
32,The aburi rolls here are spectacular. The bala...,1
11,"Just go. Takes a bit of finding, but well wor...",1
23,When the normal Chirashi bowl was served to me...,1
5,"I was taken here for dinner on a Monday night,...",1
34,"Cosy,awesome food and friendly chef! what more...",1
15,I'd like to give 4.5 stars. \nWe had Daikon sa...,1
6,Nice place for quality sushi. Business is good...,-1
10,OMG!!!!! This has to be the best find for sus...,1


About one in ten in the test set, looks correctly stratified.

## Naive Bayes Classifier

In [40]:
# Convert to dictionary format because apparently NLTK requires this
train_pos = [[row["review"], 1] for idx, row in df_train.iterrows() if row["sentiment"]==1]
train_neg = [[row["review"], -1] for idx, row in df_train.iterrows() if row["sentiment"]==-1]

test_pos = [[row["review"], 1] for idx, row in df_test.iterrows() if row["sentiment"]==1]
test_neg = [[row["review"], -1] for idx, row in df_test.iterrows() if row["sentiment"]==-1]

In [41]:
trainset = train_pos + train_neg
train_tokenized = [[wt(x), c] for x,c in trainset] # may need to introduce some pre-processing at this stage for better results

def word_feats(words):
    return dict([(word, True) for word in words])
train_featureset = [(word_feats(d), c) for (d,c) in train_tokenized] 

In [45]:
## Naive Bayes Rule using nltk
classifier_nb = NaiveBayesClassifier.train(train_featureset)
#print("Accuracy :" +str(accuracy(classifier_nb, test_featureset)))
classifier_nb.show_most_informative_features(10)

## Preparing the data first 
train_nolab = [t[0] for t in trainset]
train_lab = [t[1] for t in trainset]

# Preparing test set in same format
testset = test_pos + test_neg
test_nolab = [t[0] for t in testset]
test_lab = [t[1] for t in testset]

# Create your tf-idf function
vectorizer = TfidfVectorizer(max_df=0.7, min_df=3, use_idf=True) # modified this
train_vectors = vectorizer.fit_transform(train_nolab)
test_vectors = vectorizer.transform(test_nolab)

## train Naive Bayes Rule using sklearn
clf = MultinomialNB().fit(train_vectors, train_lab)

predNB = clf.predict(train_vectors)
pred = list(predNB)
cm1=pd.crosstab( pd.Series(train_lab), pd.Series(pred), rownames= ['actuals'], colnames=['pred'],margins=True)
cm1

Most Informative Features
                     way = True               -1 : 1      =     12.8 : 1.0
                  dishes = True               -1 : 1      =      9.2 : 1.0
                     any = True               -1 : 1      =      9.2 : 1.0
                  people = True               -1 : 1      =      9.2 : 1.0
                 outside = True               -1 : 1      =      5.5 : 1.0
               expensive = True               -1 : 1      =      5.5 : 1.0
                   kinda = True               -1 : 1      =      5.5 : 1.0
                    cold = True               -1 : 1      =      5.5 : 1.0
                  either = True               -1 : 1      =      5.5 : 1.0
                       4 = True               -1 : 1      =      5.5 : 1.0


pred,1,All
actuals,,
-1,3,3
1,21,21
All,24,24


May need some preprocessing. See '4' as an informative feature. Makes sense? 

Now test on the test set

In [52]:
predNB = clf.predict(test_vectors)
pred = list(predNB)
cm1=pd.crosstab( pd.Series(test_lab), pd.Series(pred), rownames= ['actuals'], colnames=['pred'],margins=True)
cm1


pred,1,All
actuals,,
-1,1,1
1,10,10
All,11,11


In [51]:
print (classification_report(pred,  test_lab))

             precision    recall  f1-score   support

         -1       0.00      0.00      0.00         0
          1       1.00      0.91      0.95        11

avg / total       1.00      0.91      0.95        11



/Users/pierlim/anaconda/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


## SVM Classifier

In [53]:
from sklearn.svm import SVC

# SVM Classifier from sklearn
def train_svm(X, y):
    """
    Create and train the Support Vector Machine.
    """
    svm = SVC(C=10000.0, gamma='auto', kernel='rbf')
    svm.fit(X, y)
    return svm

classifier_svm = train_svm(train_vectors, train_lab)  # training the SVM model
predSVM = classifier_svm.predict(test_vectors) 
pred_svm = list(predSVM)

In [55]:
cm6=pd.crosstab( pd.Series(test_lab), pd.Series(pred_svm), rownames= ['actuals'], colnames=['pred'],margins=True)
cm6


pred,1,All
actuals,,
-1,1,1
1,10,10
All,11,11


In [56]:
print (classification_report(pred_svm,  test_lab))

             precision    recall  f1-score   support

         -1       0.00      0.00      0.00         0
          1       1.00      0.91      0.95        11

avg / total       1.00      0.91      0.95        11



/Users/pierlim/anaconda/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
